# RAS Commander Project Initialization

This notebook demonstrates how to initialize and work with HEC-RAS projects using the `ras-commander` library. You'll learn how to:

1. Set up and configure the RAS Commander environment
2. Download and extract example HEC-RAS projects
3. Initialize HEC-RAS projects using the global `ras` object
4. Initialize multiple HEC-RAS projects using custom RAS objects
5. Access various project components (plans, geometries, flows, boundaries)

## Key Concepts

- **RasPrj Objects**: Represent HEC-RAS projects with access to plans, geometries, flows, etc.
- **Global `ras` object**: A singleton instance for simple, single-project scripts
- **Custom RAS Objects**: Independent instances for multi-project workflows
- **Project Initialization**: Process of connecting to HEC-RAS projects
- **Project Components**: Structured access to plans, geometries, and flow files

Let's start by importing the necessary libraries.

## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [1]:
# 1. Install ras-commander from pip (uncomment to install if needed)
!pip install ras-commander
# This installs ras-commander and all dependencies

In [2]:
# 2. Import all required modules

# Import all ras-commander modules
from ras_commander import *

# Import the required libraries for this notebook
import os
from pathlib import Path
import pandas as pd
from IPython import display

## RAS Commander: Core Concepts

RAS Commander is a Python library that provides tools for automating HEC-RAS tasks. It's built with several key design principles:

1. **Project-Centric Architecture**: Everything revolves around HEC-RAS projects
2. **Two RAS Object Approaches**:
   - **Global `ras` Object**: A singleton for simple scripts
   - **Custom RAS Objects**: Multiple ras project instances for complex workflows
3. **Comprehensive Project Representation**: Each RAS object includes DataFrames for plans, geometries, flows, and boundaries
4. **Logging**: Built-in logging to track operations and debug issues
5. **HDF Support**: Specialized functions for HDF file access (plan results, geometry, etc.)

Let's explore these concepts in practice.

## Downloading Example HEC-RAS Projects

RAS Commander includes a utility to download and extract example HEC-RAS projects. These are useful for learning and testing:

In [3]:
# Extract specific projects we'll use in this tutorial
# This will download them if not present and extract them to the example_projects folder
extracted_paths = RasExamples.extract_project(["Balde Eagle Creek", "BaldEagleCrkMulti2D", "Muncie"])
print(extracted_paths)

2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - Found zip file: d:\GitHub\ras-commander\examples\Example_Projects_6_6.zip
2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - Extracting project 'Balde Eagle Creek'
2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - Project 'Balde Eagle Creek' already exists. Deleting existing folder...
2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - Existing folder for project 'Balde Eagle Creek' has been deleted.
2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - Successfully extracted project 'Balde Eagle Creek' to d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek
2025-04-08 11:33:03 - ras_commander.RasExamples - INFO - ----- Ras

[WindowsPath('d:/GitHub/ras-commander/examples/example_projects/Balde Eagle Creek'), WindowsPath('d:/GitHub/ras-commander/examples/example_projects/BaldEagleCrkMulti2D'), WindowsPath('d:/GitHub/ras-commander/examples/example_projects/Muncie')]


# Get Paths for Extracted Example Projects

In [4]:
# Get the parent directory of the first extracted path as our examples directory
examples_dir = extracted_paths[0].parent
print(f"Examples directory: {examples_dir}")


# Define paths to the extracted projects
bald_eagle_path = examples_dir / "Balde Eagle Creek"
multi_2d_path = examples_dir / "BaldEagleCrkMulti2D"
muncie_path = examples_dir / "Muncie"

# Verify the paths exist
for path in [bald_eagle_path, multi_2d_path, muncie_path]:
    print(f"Path {path} exists: {path.exists()}")

Examples directory: d:\GitHub\ras-commander\examples\example_projects
Path d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek exists: True
Path d:\GitHub\ras-commander\examples\example_projects\BaldEagleCrkMulti2D exists: True
Path d:\GitHub\ras-commander\examples\example_projects\Muncie exists: True


## Utility Function to Print RAS Object Data

Let's create a utility function to help us explore the contents of RAS objects:

In [5]:
def print_ras_object_data(ras_obj, project_name):
    """Prints comprehensive information about a RAS object"""
    print(f"\n{project_name} Data:")
    print("=" * 50)
    print(f"Project Name: {ras_obj.get_project_name()}")
    print(f"Project Folder: {ras_obj.project_folder}")
    print(f"PRJ File: {ras_obj.prj_file}")
    print(f"HEC-RAS Executable Path: {ras_obj.ras_exe_path}")
    
    print("\nPlan Files DataFrame (ras.plan_df):")
    with pd.option_context('display.max_columns', None):
        display.display(ras_obj.plan_df)
    
    print("\nFlow Files DataFrame:")
    display.display(ras_obj.flow_df)
    
    print("\nUnsteady Flow Files DataFrame (ras.unsteady_df):")
    display.display(ras_obj.unsteady_df)
    
    print("\nGeometry Files DataFrame (ras.geom_df):")
    display.display(ras_obj.geom_df)
    
    print("\nHDF Entries DataFrame (ras.get_hdf_entries()):")
    display.display(ras_obj.get_hdf_entries())
    
    print("\nBoundary Conditions DataFrame (ras.boundaries_df):")
    display.display(ras_obj.boundaries_df)

## Approach 1: Using the Global `ras` Object

The global `ras` object is a singleton instance that persists throughout your script. It's ideal for simple scripts working with a single project.

Key characteristics:
- It's available as `ras` immediately after import
- It's initialized via `init_ras_project()` without saving the return value
- It provides access to all project data through the global `ras` variable
- It's simple to use but can be problematic in complex scenarios

Let's initialize it with the Bald Eagle Creek project:

In [6]:
# Initialize the global ras object with Bald Eagle Creek project
# Note: This updates the global 'ras' object visible throughout the script
# Parameters:
#   - project_folder: Path to the HEC-RAS project folder (required)
#   - ras_version: HEC-RAS version (e.g. "6.5") or path to Ras.exe (required first time)

init_ras_project(bald_eagle_path, "6.5")
print(f"The global 'ras' object is now initialized with the {ras.project_name} project")

2025-04-08 11:33:08 - ras_commander.RasPrj - INFO - Initializing global 'ras' object via init_ras_project function.
2025-04-08 11:33:08 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.rasmap
2025-04-08 11:33:08 - ras_commander.RasPrj - INFO - Project initialized. ras_object project folder: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek


The global 'ras' object is now initialized with the BaldEagle project


In [7]:
# Explore the global ras object with our utility function
print_ras_object_data(ras, "Global RAS Object (Bald Eagle Creek)")


Global RAS Object (Bald Eagle Creek) Data:
Project Name: BaldEagle
Project Folder: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek
PRJ File: D:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.prj
HEC-RAS Executable Path: C:\Program Files (x86)\HEC\HEC-RAS\6.5\Ras.exe

Plan Files DataFrame (ras.plan_df):


plan_number unsteady_number geometry_number                     Plan Title  \
0          01              02              01  Unsteady with Bridges and Dam   
1          02            None              01                Steady Flow Run   

  Program Version Short Identifier                  Simulation Date  \
0            5.00     UnsteadyFlow    18FEB1999,0000,24FEB1999,0500   
1             NaN        SteadyRun  02/18/1999,0000,02/24/1999,0500   

  Computation Interval Mapping Interval Run HTab Run UNet Run Sediment  \
0                 2MIN            1HOUR        1        1            0   
1                 2MIN              NaN        1        1          NaN   

  Run PostProcess Run WQNet UNET Use Existing IB Tables  UNET D1 Cores  \
0               1         0                          -1            0.0   
1               1       NaN                         NaN            NaN   

   UNET D2 Cores PS Cores DSS File Friction Slope Method HDF_Results_Path  \
0            0.0     None      dss                     2             None   
1            NaN     None      dss                     1             None   

  Geom File                                          Geom Path Flow File  \
0        01  d:\GitHub\ras-commander\examples\example_proje...        02   
1        01  d:\GitHub\ras-commander\examples\example_proje...        02   

                                           Flow Path  \
0  d:\GitHub\ras-commander\examples\example_proje...   
1  d:\GitHub\ras-commander\examples\example_proje...   

                                           full_path  
0  d:\GitHub\ras-commander\examples\example_proje...  
1  d:\GitHub\ras-commander\examples\example_proje...


Flow Files DataFrame:


flow_number                                          full_path  \
0          02  d:\GitHub\ras-commander\examples\example_proje...   
1          01  d:\GitHub\ras-commander\examples\example_proje...   

  unsteady_number geometry_number  
0            None            None  
1            None            None


Unsteady Flow Files DataFrame (ras.unsteady_df):


...

geom_file geom_number                                          full_path  \
0       g01          01  d:\GitHub\ras-commander\examples\example_proje...   

                                            hdf_path  
0  d:\GitHub\ras-commander\examples\example_proje...


HDF Entries DataFrame (ras.get_hdf_entries()):


Empty DataFrame
Columns: [plan_number, unsteady_number, geometry_number, Plan Title, Program Version, Short Identifier, Simulation Date, Computation Interval, Mapping Interval, Run HTab, Run UNet, Run Sediment, Run PostProcess, Run WQNet, UNET Use Existing IB Tables, UNET D1 Cores, UNET D2 Cores, PS Cores, DSS File, Friction Slope Method, HDF_Results_Path, Geom File, Geom Path, Flow File, Flow Path, full_path]
Index: []

[0 rows x 26 columns]


Boundary Conditions DataFrame (ras.boundaries_df):


unsteady_number  boundary_condition_number river_reach_name river_station  \
0              02                          1       Bald Eagle       Loc Hav   
1              02                          2       Bald Eagle       Loc Hav   
2              02                          3       Bald Eagle       Loc Hav   

  storage_area_name pump_station_name          bc_type  hydrograph_type  \
0          138154.4                    Flow Hydrograph  Flow Hydrograph   
1             81500                       Gate Opening             None   
2           659.942                            Unknown             None   

  Interval DSS Path  ...         Flow Title Program Version Use Restart  \
0    1HOUR           ...  Flow Hydrograph 2            6.30           0   
1      NaN      NaN  ...  Flow Hydrograph 2            6.30           0   
2      NaN           ...  Flow Hydrograph 2            6.30           0   

  Precipitation Mode       Wind Mode  Met BC=Precipitation|Mode  \
0            Disable  No Wind Forces                       None   
1            Disable  No Wind Forces                       None   
2            Disable  No Wind Forces                       None   

  Met BC=Evapotranspiration|Mode Met BC=Precipitation|Expanded View  \
0                           None                                  0   
1                           None                                  0   
2                           None                                  0   

  Met BC=Precipitation|Constant Units Met BC=Precipitation|Gridded Source  
0                               mm/hr                                 DSS  
1                               mm/hr                                 DSS  
2                               mm/hr                                 DSS  

[3 rows x 29 columns]

### Understanding the RAS Object Structure

Each RAS object contains several important components:

1. **Project Metadata**:
   - `project_name`: Name of the HEC-RAS project
   - `project_folder`: Directory containing project files
   - `prj_file`: Path to the main .prj file
   - `ras_exe_path`: Path to the HEC-RAS executable

2. **Project DataFrames**:
   - `plan_df`: Information about all plan files (.p*)
   - `flow_df`: Information about all steady flow files (.f*)
   - `unsteady_df`: Information about all unsteady flow files (.u*)
   - `geom_df`: Information about all geometry files (.g*)
   - `boundaries_df`: Information about all boundary conditions

3. **Methods for Data Access**:
   - `get_plan_entries()`: Get plan file information
   - `get_flow_entries()`: Get flow file information
   - `get_unsteady_entries()`: Get unsteady flow file information 
   - `get_geom_entries()`: Get geometry file information
   - `get_hdf_entries()`: Get HDF file paths for result files
   - `get_boundary_conditions()`: Get boundary condition details

Let's see how to access specific information from these components:

In [8]:
# Get the first plan's details
if not ras.plan_df.empty:
    first_plan = ras.plan_df.iloc[0]
    print(f"First plan number: {first_plan['plan_number']}")
    print(f"Plan path: {first_plan['full_path']}")
    
    # Get the geometry file for this plan
    geom_id = first_plan.get('Geom File', '').replace('g', '')
    if geom_id:
        geom_info = ras.geom_df[ras.geom_df['geom_number'] == geom_id]
        if not geom_info.empty:
            print(f"Geometry file: {geom_info.iloc[0]['full_path']}")
    
    # Get the HDF results file for this plan (if exists)
    if 'HDF_Results_Path' in first_plan and first_plan['HDF_Results_Path']:
        print(f"Results file: {first_plan['HDF_Results_Path']}")
else:
    print("No plans found in the project.")

First plan number: 01
Plan path: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.p01
Geometry file: d:\GitHub\ras-commander\examples\example_projects\Balde Eagle Creek\BaldEagle.g01


### Working with Boundary Conditions

Boundary conditions define the inputs and outputs of your model. Let's see how to access boundary condition information:

In [9]:
# View the boundary conditions DataFrame
ras.boundaries_df 

['  unsteady_number  boundary_condition_number river_reach_name river_station  \\\n', '0              02                          1       Bald Eagle       Loc Hav   \n', '1              02                          2       Bald Eagle       Loc Hav   \n', '2              02                          3       Bald Eagle       Loc Hav   \n', '\n', '  storage_area_name pump_station_name          bc_type  hydrograph_type  \\\n', '0          138154.4                    Flow Hydrograph  Flow Hydrograph   \n', '1             81500                       Gate Opening             None   \n', '2           659.942                            Unknown             None   \n', '\n', '  Interval DSS Path  ...         Flow Title Program Version Use Restart  \\\n', '0    1HOUR           ...  Flow Hydrograph 2            6.30           0   \n', '1      NaN      NaN  ...  Flow Hydrograph 2            6.30           0   \n', '2      NaN           ...  Flow Hydrograph 2            6.30           0   \n', '\n', '  Precipitation Mode       Wind Mode  Met BC=Precipitation|Mode  \\\n', '0            Disable  No Wind Forces                       None   \n', '1            Disable  No Wind Forces                       None   \n', '2            Disable  No Wind Forces                       None   \n', '\n', '  Met BC=Evapotranspiration|Mode Met BC=Precipitation|Expanded View  \\\n', '0                           None                                  0   \n', '1                           None                                  0   \n', '2                           None                                  0   \n', '\n', '  Met BC=Precipitation|Constant Units Met BC=Precipitation|Gridded Source  \n', '0                               mm/hr                                 DSS  \n', '1                               mm/hr                                 DSS  \n', '2                               mm/hr                                 DSS  \n', '\n', '[3 rows x 29 columns]']

## Approach 2: Using Custom RAS Objects

For more complex scripts or when working with multiple projects, it's better to create and use separate RAS objects. This approach:

- Creates independent RAS objects for each project
- Avoids overwriting the global `ras` object
- Provides clearer separation between projects
- Allows working with multiple projects simultaneously
- Requires saving the return value from `init_ras_project()`

Let's initialize multiple projects with custom RAS objects:

In [10]:
# Initialize multiple project instances with custom RAS objects
# Note: This also updates the global 'ras' object each time, but we'll use the custom instances
# Parameters remain the same as before
multi_2d_project = RasPrj()
init_ras_project(multi_2d_path, "6.5", ras_object=multi_2d_project)
print(f"\nMulti2D project initialized with its own RAS object")

muncie_project = RasPrj()
init_ras_project(muncie_path, "6.5", ras_object=muncie_project)
print(f"\nMuncie project initialized with its own RAS object")

# Note that the global 'ras' object now points to the Muncie project
# The global 'ras' object gets overwritten every time a project is initialized ,
print(f"\nGlobal 'ras' object now points to: {ras.project_name} since it was the last one initialized.  Avoid the global object when using multiple projects.")

2025-04-08 11:33:08 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: d:\GitHub\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.rasmap
2025-04-08 11:33:08 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: d:\GitHub\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.rasmap
2025-04-08 11:33:08 - ras_commander.RasPrj - INFO - Global 'ras' object also updated to match the new project.
2025-04-08 11:33:08 - ras_commander.RasPrj - INFO - Project initialized. ras_object project folder: d:\GitHub\ras-commander\examples\example_projects\BaldEagleCrkMulti2D
2025-04-08 11:33:09 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: d:\GitHub\ras-commander\examples\example_projects\Muncie\Muncie.rasmap
2025-04-08 11:33:09 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: d:\GitHub\ras-commander\examples\example_projects\Muncie\Muncie.rasmap
2025-04-08 11:33:09 - ras_commander.Ras


Multi2D project initialized with its own RAS object

Muncie project initialized with its own RAS object

Global 'ras' object now points to: Muncie since it was the last one initialized.  Avoid the global object when using multiple projects.


### Exploring Multiple Projects

Now we have three RAS objects:
- `multi_2d_project`: Our custom object for the Multi2D project
- `muncie_project`: Our custom object for the Muncie project
- `ras`: The global object (which now points to Muncie)

Let's examine the Multi2D project:

In [11]:
display.display(multi_2d_project.plan_df)

plan_number unsteady_number geometry_number  \
0           13              07              06   
1           15              12              08   
2           17              09              10   
3           18              10              11   
4           19              11              12   
5           03              13              09   
6           04              01              13   
7           02              01              01   
8           01              01              01   
9           05              02              03   
10          06              03              09   

                                 Plan Title Program Version  \
0                   PMF with Multi 2D Areas            5.10   
1               1d-2D Dambreak Refined Grid            5.10   
2                           2D to 1D No Dam            5.00   
3                              2D to 2D Run            5.00   
4                    SA to 2D Dam Break Run            5.00   
5   Single 2D Area - Internal Dam Structure            5.04   
6   SA to 2D Area Conn - 2D Levee Structure            5.00   
7                  SA to Detailed 2D Breach            5.10   
8              SA to Detailed 2D Breach FEQ            5.03   
9           Single 2D area with Bridges FEQ            5.10   
10            Gridded Precip - Infiltration            6.00   

            Short Identifier                Simulation Date  \
0               PMF Multi 2D  01JAN1999,1200,04JAN1999,1200   
1         1D-2D Refined Grid  01JAN1999,1200,04JAN1999,1200   
2            2D to 1D No Dam  01JAN1999,1200,06JAN1999,1200   
3               2D to 2D Run  01JAN1999,1200,04JAN1999,1200   
4         SA to 2D Dam Break  01JAN1999,1200,04JAN1999,1200   
5                  Single 2D  01JAN1999,1200,04JAN1999,1200   
6             2D Levee Struc  01JAN1999,1200,04JAN1999,1200   
7             SA-2D Det Brch  01JAN1999,1200,04JAN1999,1200   
8              SA-2D Det FEQ  01JAN1999,1200,04JAN1999,1200   
9      Single 2D Bridges FEQ  01JAN1999,1200,04JAN1999,1200   
10  Grid Precip Infiltration  09SEP2018,0000,14SEP2018,0000   

   Computation Interval Mapping Interval Run HTab  ... DSS File  \
0                 30SEC            30MIN        1  ...      dss   
1                 20SEC             5MIN        1  ...      dss   
2                  1MIN             5MIN        1  ...      dss   
3                 20SEC             5MIN        1  ...      dss   
4                 20SEC            10MIN        1  ...      dss   
5                 30SEC            10MIN        1  ...      dss   
6                 20SEC             5MIN        1  ...      dss   
7                 10SEC             5MIN        1  ...      dss   
8                  5SEC             5MIN        1  ...      dss   
9                  5SEC            10MIN       -1  ...      dss   
10                20SEC            10MIN       -1  ...      dss   

   Friction Slope Method UNET D2 SolverType UNET D2 Name HDF_Results_Path  \
0                      1   Pardiso (Direct)          193             None   
1                      1                NaN  BaldEagleCr             None   
2                      1                NaN   Upstream2D             None   
3                      1                NaN  BaldEagleCr             None   
4                      1                NaN  BaldEagleCr             None   
5                      1                NaN  BaldEagleCr             None   
6                      1                NaN  BaldEagleCr             None   
7                      1   Pardiso (Direct)  BaldEagleCr             None   
8                      1                NaN  BaldEagleCr             None   
9                      1   PARDISO (Direct)  BaldEagleCr             None   
10                     1   Pardiso (Direct)  BaldEagleCr             None   

   Geom File                                          Geom Path  Flow File  \
0         06  d:\GitHub\ras-commander\examples\example_proje...         07   
1 

In [12]:
# Examine the Multi2D project
print_ras_object_data(multi_2d_project, "Multi2D Project")


Multi2D Project Data:
Project Name: BaldEagleDamBrk
Project Folder: d:\GitHub\ras-commander\examples\example_projects\BaldEagleCrkMulti2D
PRJ File: D:\GitHub\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.prj
HEC-RAS Executable Path: C:\Program Files (x86)\HEC\HEC-RAS\6.5\Ras.exe

Plan Files DataFrame (ras.plan_df):


plan_number unsteady_number geometry_number  \
0           13              07              06   
1           15              12              08   
2           17              09              10   
3           18              10              11   
4           19              11              12   
5           03              13              09   
6           04              01              13   
7           02              01              01   
8           01              01              01   
9           05              02              03   
10          06              03              09   

                                 Plan Title Program Version  \
0                   PMF with Multi 2D Areas            5.10   
1               1d-2D Dambreak Refined Grid            5.10   
2                           2D to 1D No Dam            5.00   
3                              2D to 2D Run            5.00   
4                    SA to 2D Dam Break Run            5.00   
5   Single 2D Area - Internal Dam Structure            5.04   
6   SA to 2D Area Conn - 2D Levee Structure            5.00   
7                  SA to Detailed 2D Breach            5.10   
8              SA to Detailed 2D Breach FEQ            5.03   
9           Single 2D area with Bridges FEQ            5.10   
10            Gridded Precip - Infiltration            6.00   

            Short Identifier                Simulation Date  \
0               PMF Multi 2D  01JAN1999,1200,04JAN1999,1200   
1         1D-2D Refined Grid  01JAN1999,1200,04JAN1999,1200   
2            2D to 1D No Dam  01JAN1999,1200,06JAN1999,1200   
3               2D to 2D Run  01JAN1999,1200,04JAN1999,1200   
4         SA to 2D Dam Break  01JAN1999,1200,04JAN1999,1200   
5                  Single 2D  01JAN1999,1200,04JAN1999,1200   
6             2D Levee Struc  01JAN1999,1200,04JAN1999,1200   
7             SA-2D Det Brch  01JAN1999,1200,04JAN1999,1200   
8              SA-2D Det FEQ  01JAN1999,1200,04JAN1999,1200   
9      Single 2D Bridges FEQ  01JAN1999,1200,04JAN1999,1200   
10  Grid Precip Infiltration  09SEP2018,0000,14SEP2018,0000   

   Computation Interval Mapping Interval Run HTab Run UNet Run Sediment  \
0                 30SEC            30MIN        1        1            0   
1                 20SEC             5MIN        1        1            0   
2                  1MIN             5MIN        1        1            0   
3                 20SEC             5MIN        1        1            0   
4                 20SEC            10MIN        1        1            0   
5                 30SEC            10MIN        1        1            0   
6                 20SEC             5MIN        1        1            0   
7                 10SEC             5MIN        1        1            0   
8                  5SEC             5MIN        1        1            0   
9                  5SEC            10MIN       -1       -1            0   
10                20SEC            10MIN       -1       -1            0   

   Run PostProcess Run WQNet UNET Use Existing IB Tables UNET 1D Methodology  \
0                1         0                          -1   Finite Difference   
1                1         0                          -1   Finite Difference   
2                1         0                          -1                 NaN   
3                1         0                          -1                 NaN   
4                1         0                          -1                 NaN   
5                1         0                          -1   Finite Difference   
6                1         0                          -1                 NaN   
7                1         0                          -1   Finite Difference   
8                1         0                          -1                 NaN   
9               -1         0                          -1   Finite Difference   
10              -1         0                          -1   Finite Difference   

    UNET D1 Cores  UNET D2


Flow Files DataFrame:


""
[... additional rows truncated ...]



Unsteady Flow Files DataFrame (ras.unsteady_df):


...

geom_file geom_number                                          full_path  \
0       g06          06  d:\GitHub\ras-commander\examples\example_proje...   
1       g08          08  d:\GitHub\ras-commander\examples\example_proje...   
2       g10          10  d:\GitHub\ras-commander\examples\example_proje...   
3       g11          11  d:\GitHub\ras-commander\examples\example_proje...   
4       g12          12  d:\GitHub\ras-commander\examples\example_proje...   
5       g09          09  d:\GitHub\ras-commander\examples\example_proje...   
6       g13          13  d:\GitHub\ras-commander\examples\example_proje...   
7       g01          01  d:\GitHub\ras-commander\examples\example_proje...   
8       g03          03  d:\GitHub\ras-commander\examples\example_proje...   
9       g02          02  d:\GitHub\ras-commander\examples\example_proje...   

                                            hdf_path  
0  d:\GitHub\ras-commander\examples\example_proje...  
1  d:\GitHub\ras-commander\examples\example_proje...  
2  d:\GitHub\ras-commander\examples\example_proje...  
3  d:\GitHub\ras-commander\examples\example_proje...  
4  d:\GitHub\ras-commander\examples\example_proje...  
5  d:\GitHub\ras-commander\examples\example_proje...  
6  d:\GitHub\ras-commander\examples\example_proje...  
7  d:\GitHub\ras-commander\examples\example_proje...  
8  d:\GitHub\ras-commander\examples\example_proje...  
9  d:\GitHub\ras-commander\examples\example_proje...


HDF Entries DataFrame (ras.get_hdf_entries()):


Empty DataFrame
Columns: [plan_number, unsteady_number, geometry_number, Plan Title, Program Version, Short Identifier, Simulation Date, Computation Interval, Mapping Interval, Run HTab, Run UNet, Run Sediment, Run PostProcess, Run WQNet, UNET Use Existing IB Tables, UNET 1D Methodology, UNET D1 Cores, UNET D2 Cores, PS Cores, DSS File, Friction Slope Method, UNET D2 SolverType, UNET D2 Name, HDF_Results_Path, Geom File, Geom Path, Flow File, Flow Path, full_path]
Index: []

[0 rows x 29 columns]


Boundary Conditions DataFrame (ras.boundaries_df):


unsteady_number  boundary_condition_number river_reach_name river_station  \
0               07                          1   Bald Eagle Cr.    Lock Haven   
1               07                          2   Bald Eagle Cr.    Lock Haven   
2               07                          3   Bald Eagle Cr.    Lock Haven   
3               07                          4   Bald Eagle Cr.    Lock Haven   
4               07                          5   Bald Eagle Cr.    Lock Haven   
5               07                          6   Bald Eagle Cr.    Lock Haven   
6               07                          7   Bald Eagle Cr.    Lock Haven   
7               07                          8   Bald Eagle Cr.    Lock Haven   
8               07                          9   Bald Eagle Cr.    Lock Haven   
9               07                         10   Bald Eagle Cr.    Lock Haven   
10              08                          1   Bald Eagle Cr.    Lock Haven   
11              08                          2   Bald Eagle Cr.    Lock Haven   
12              08                          3   Bald Eagle Cr.    Lock Haven   
13              08                          4   Bald Eagle Cr.    Lock Haven   
14              08                          5   Bald Eagle Cr.    Lock Haven   
15              08                          6   Bald Eagle Cr.    Lock Haven   
16              08                          7   Bald Eagle Cr.    Lock Haven   
17              08                          8                                  
18              08                          9   Bald Eagle Cr.    Lock Haven   
19              09                          1   Bald Eagle Cr.    Lock Haven   
20              09                          2   Bald Eagle Cr.    Lock Haven   
21              09                          3                                  
22              10                          1                                  
23              10                          2                                  
24              10                          3                                  
25              10                          4                                  
26              11                          1                                  
27              11                          2                                  
28              11                          3                                  
29              11                          4                                  
30              12                          1   Bald Eagle Cr.    Lock Haven   
31              12                          2   Bald Eagle Cr.    Lock Haven   
32              12                          3                                  
33              12                          4                                  
34              13                          1                                  
35              13                          2                                  
36              13                          3                                  
37              13                          4                                  
38              01                          1                                  
39              01                          2                                  
40              01                          3                                  
41              01                          4                                  
42              02                          1                                  
43              02                          2                                  
44              02                          3                                  
45              02                          4                                  
46              02                          5                                  
47              03                          1                                  
48              03                          2                                  
49 

In [13]:
# Examine the Muncie project
print_ras_object_data(muncie_project, "Muncie Project")


Muncie Project Data:
Project Name: Muncie
Project Folder: d:\GitHub\ras-commander\examples\example_projects\Muncie
PRJ File: D:\GitHub\ras-commander\examples\example_projects\Muncie\Muncie.prj
HEC-RAS Executable Path: C:\Program Files (x86)\HEC\HEC-RAS\6.5\Ras.exe

Plan Files DataFrame (ras.plan_df):


plan_number unsteady_number geometry_number  \
0          01              01              01   
1          03              01              02   
2          04              01              04   

                                 Plan Title Program Version  \
0                  Unsteady Multi  9-SA run            5.00   
1            Unsteady Run with 2D 50ft Grid            5.10   
2  Unsteady Run with 2D 50ft User n Value R            5.10   

      Short Identifier                Simulation Date Computation Interval  \
0                9-SAs  02JAN1900,0000,02JAN1900,2400                15SEC   
1         2D 50ft Grid  02JAN1900,0000,02JAN1900,2400                10SEC   
2  50ft User n Regions  02JAN1900,0000,02JAN1900,2400                10SEC   

  Mapping Interval Run HTab Run UNet Run Sediment Run PostProcess Run WQNet  \
0             5MIN        1        1            0               1         0   
1             5MIN       -1       -1            0              -1         0   
2             5MIN        1        1            0               1         0   

  UNET Use Existing IB Tables UNET 1D Methodology  UNET D1 Cores  \
0                          -1                 NaN              0   
1                          -1   Finite Difference              0   
2                          -1   Finite Difference              0   

   UNET D2 Cores PS Cores DSS File Friction Slope Method UNET D2 SolverType  \
0              0     None      dss                     1                NaN   
1              4     None      dss                     1   Pardiso (Direct)   
2              6     None      dss                     1   Pardiso (Direct)   

       UNET D2 Name HDF_Results_Path Geom File  \
0               NaN             None        01   
1  2D Interior Area             None        02   
2  2D Interior Area             None        04   

                                           Geom Path Flow File  \
0  d:\GitHub\ras-commander\examples\example_proje...        01   
1  d:\GitHub\ras-commander\examples\example_proje...        01   
2  d:\GitHub\ras-commander\examples\example_proje...        01   

                                           Flow Path  \
0  d:\GitHub\ras-commander\examples\example_proje...   
1  d:\GitHub\ras-commander\examples\example_proje...   
2  d:\GitHub\ras-commander\examples\example_proje...   

                                           full_path  
0  d:\GitHub\ras-commander\examples\example_proje...  
1  d:\GitHub\ras-commander\examples\example_proje...  
2  d:\GitHub\ras-commander\examples\example_proje...


Flow Files DataFrame:


flow_number                                          full_path  \
0          01  d:\GitHub\ras-commander\examples\example_proje...   

  unsteady_number geometry_number  
0            None            None


Unsteady Flow Files DataFrame (ras.unsteady_df):


...

geom_file geom_number                                          full_path  \
0       g01          01  d:\GitHub\ras-commander\examples\example_proje...   
1       g02          02  d:\GitHub\ras-commander\examples\example_proje...   
2       g04          04  d:\GitHub\ras-commander\examples\example_proje...   

                                            hdf_path  
0  d:\GitHub\ras-commander\examples\example_proje...  
1  d:\GitHub\ras-commander\examples\example_proje...  
2  d:\GitHub\ras-commander\examples\example_proje...


HDF Entries DataFrame (ras.get_hdf_entries()):


Empty DataFrame
Columns: [plan_number, unsteady_number, geometry_number, Plan Title, Program Version, Short Identifier, Simulation Date, Computation Interval, Mapping Interval, Run HTab, Run UNet, Run Sediment, Run PostProcess, Run WQNet, UNET Use Existing IB Tables, UNET 1D Methodology, UNET D1 Cores, UNET D2 Cores, PS Cores, DSS File, Friction Slope Method, UNET D2 SolverType, UNET D2 Name, HDF_Results_Path, Geom File, Geom Path, Flow File, Flow Path, full_path]
Index: []

[0 rows x 29 columns]


Boundary Conditions DataFrame (ras.boundaries_df):


unsteady_number  boundary_condition_number river_reach_name river_station  \
0              01                          1            White        Muncie   
1              01                          2            White        Muncie   

  storage_area_name pump_station_name          bc_type  hydrograph_type  \
0          15696.24                    Flow Hydrograph  Flow Hydrograph   
1          237.6455                       Normal Depth             None   

  Interval DSS Path  ...                                  hydrograph_values  \
0    1HOUR           ...  [13500, 14000, 14500, 15000, 15500, 16000, 165...   
1      NaN      NaN  ...                                                NaN   

                                           full_path geometry_number  \
0  d:\GitHub\ras-commander\examples\example_proje...            None   
1  d:\GitHub\ras-commander\examples\example_proje...            None   

                 Flow Title Program Version  Use Restart Precipitation Mode  \
0  Flow Boundary Conditions            6.30            0            Disable   
1  Flow Boundary Conditions            6.30            0            Disable   

        Wind Mode Met BC=Precipitation|Expanded View  \
0  No Wind Forces                                  0   
1  No Wind Forces                                  0   

  Met BC=Precipitation|Gridded Source  
0                                 DSS  
1                                 DSS  

[2 rows x 26 columns]

### Comparing Projects

Let's compare some key metrics of the two projects:

In [14]:
# Create a comparison table of the two projects
comparison_data = {
    'Project Name': [multi_2d_project.project_name, muncie_project.project_name],
    'Number of Plans': [len(multi_2d_project.plan_df), len(muncie_project.plan_df)],
    'Number of Geometries': [len(multi_2d_project.geom_df), len(muncie_project.geom_df)],
    'Number of Flow Files': [len(multi_2d_project.flow_df), len(muncie_project.flow_df)],
    'Number of Unsteady Files': [len(multi_2d_project.unsteady_df), len(muncie_project.unsteady_df)],
    'Number of Boundary Conditions': [len(multi_2d_project.boundaries_df) if hasattr(multi_2d_project, 'boundaries_df') else 0, 
                                     len(muncie_project.boundaries_df) if hasattr(muncie_project, 'boundaries_df') else 0],
    'HDF Results Available': [len(multi_2d_project.get_hdf_entries()) > 0, len(muncie_project.get_hdf_entries()) > 0]
}

comparison_df = pd.DataFrame(comparison_data)
display.display(comparison_df)

Project Name  Number of Plans  Number of Geometries  \
0  BaldEagleDamBrk               11                    10   
1           Muncie                3                     3   

   Number of Flow Files  Number of Unsteady Files  \
0                     0                        10   
1                     1                         1   

   Number of Boundary Conditions  HDF Results Available  
0                             51                  False  
1                              2                  False

## RAS Commander: Best Practices

After exploring both approaches, here are some best practices for using RAS Commander:

1. **Choose Your Approach Based on Complexity**:
   - **Simple Scripts** (one project): Use the global `ras` object
   - **Complex Scripts** (multiple projects): Use custom RAS objects

2. **Be Consistent**:
   - Don't mix global and custom approaches in the same script
   - Use descriptive names for custom RAS objects

3. **Working with Project Files**:
   - Access project files through the RAS object's DataFrames
   - Use helper functions like `get_plan_path()` to resolve paths

4. **Error Handling**:
   - Always check for empty DataFrames before accessing their contents
   - Use the built-in logging to track operations

5. **Performance Considerations**:
   - For large projects, consider using the HDF classes directly
   - Cache results of expensive operations when possible

## Summary of Key Functions

- `init_ras_project(project_folder, ras_version)`: Initialize a RAS project
- `RasExamples().extract_project(project_name)`: Extract example projects
- `RasPrj.get_project_name()`: Get the name of the project
- `RasPrj.get_plan_entries()`: Get plan file information
- `RasPrj.get_flow_entries()`: Get flow file information
- `RasPrj.get_unsteady_entries()`: Get unsteady flow file information
- `RasPrj.get_geom_entries()`: Get geometry file information
- `RasPrj.get_hdf_entries()`: Get HDF result file information
- `RasPrj.get_boundary_conditions()`: Get boundary condition details
- `RasPlan.get_plan_path(plan_number)`: Get the path to a plan file
- `RasPlan.get_geom_path(geom_number)`: Get the path to a geometry file
- `RasPlan.get_flow_path(flow_number)`: Get the path to a flow file
- `RasPlan.get_unsteady_path(unsteady_number)`: Get the path to an unsteady flow file

## Next Steps

Now that you understand the basics of project initialization in RAS Commander, you can explore more advanced topics:

1. Working with HDF files for result analysis
2. Modifying plan, geometry, and flow files
3. Running HEC-RAS simulations
4. Extracting and visualizing results
5. Automating model calibration

These topics are covered in other examples and notebooks in the RAS Commander documentation.